<a href="https://colab.research.google.com/github/luuleitner/xMason/blob/main/xmasonsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# xMason can be directly installed from the GitHub repository using PIP.

!pip install git+https://github.com/luuleitner/xMason

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/luuleitner/xMason to /tmp/pip-req-build-zbt0ia0k
  Running command git clone -q https://github.com/luuleitner/xMason /tmp/pip-req-build-zbt0ia0k
     |████████████████████████████████| 51 kB 295 kB/s 
     |████████████████████████████████| 251 kB 8.4 MB/s 
  Created wheel for xMason: filename=xMason-1.0-py3-none-any.whl size=15944 sha256=6ed8495dd09d676db36867f6ac0e7738ada0eceafe1cb6945956e9473de368aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-jtyf9hp_/wheels/2b/97/35/9cc839136d62979206df40b03dcc41952315ae913e8ef6f43c
Successfully built xMason


In [2]:
# We need to import necessary packages into Colab. Don't worry to much about this cell of code

from simulation.src.data.loader import VNA_Dataloader
from simulation.src.models.simulate import simulate_xMason
from simulation.src.features.ports import frequency_spectrum

import numpy as np # array handeling
from urllib.request import urlretrieve # for downloading the sample data
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.signal import find_peaks